<a href="https://colab.research.google.com/github/semreyalcin/CNG562-FinalProject/blob/master/SEMERCI_YALCIN_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Home Credit Default Risk
---
The aim of this project is to predict if the clients will repay a loan or not.

Group project by 

Mehmet Semerci - 2152163


Selim Emre Yalçın - 2152205



##Loading Data
Downloading data from kaggle

In [0]:
!pip install -q kaggle
!pip install -q kaggle-cli

In [0]:
import json
!mkdir .kaggle
!mkdir ~/.kaggle

token = {"username":"msemerci","key":"ea86d09950c75144f8b3bb218ba348a9"}

with open('/content/.kaggle/kaggle.json', 'w') as file:
    json.dump(token, file)

In [0]:
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!kaggle config set -n path -v /content 

In [0]:
!kaggle competitions download -c home-credit-default-risk

#Reading Data
Reading only the main table which is application_train table in.

In [5]:
!mv competitions/home-credit-default-risk/application_train.csv.zip /content/
!unzip application_train.csv.zip
!rm application_train.csv.zip

Archive:  application_train.csv.zip
  inflating: application_train.csv   


In [6]:
import pandas as pd
import numpy as np

df = pd.read_csv("application_train.csv")
df.head()

,SK_ID_CURR,TARGET,NAME_CONTRACT_TYPE,CODE_GENDER,FLAG_OWN_CAR,FLAG_OWN_REALTY,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,NAME_TYPE_SUITE,NAME_INCOME_TYPE,NAME_EDUCATION_TYPE,NAME_FAMILY_STATUS,NAME_HOUSING_TYPE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,OCCUPATION_TYPE,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,WEEKDAY_APPR_PROCESS_START,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,...,LIVINGAPARTMENTS_MEDI,LIVINGAREA_MEDI,NONLIVINGAPARTMENTS_MEDI,NONLIVINGAREA_MEDI,FONDKAPREMONT_MODE,HOUSETYPE_MODE,TOTALAREA_MODE,WALLSMATERIAL_MODE,EMERGENCYSTATE_MODE,OBS_30_CNT_SOCIAL_CIRCLE,DEF_30_CNT_SOCIAL_CIRCLE,OBS_60_CNT_SOCIAL_CIRCLE,DEF_60_CNT_SOCIAL_CIRCLE,DAYS_LAST_PHONE_CHANGE,FLAG_DOCUMENT_2,FLAG_DOCUMENT_3,FLAG_DOCUMENT_4,FLAG_DOCUMENT_5,FLAG_DOCUMENT_6,FLAG_DOCUMENT_7,FLAG_DOCUMENT_8,FLAG_DOCUMENT_9,FLAG_DOCUMENT_10,FLAG_DOCUMENT_11,FLAG_DOCUMENT_12,FLAG_DOCUMENT_13,FLAG_DOCUMENT_14,FLAG_DOCUMENT_15,FLAG_DOCUMENT_16,FLAG_DOCUMENT_17,FLAG_DOCUMENT_18,FLAG_DOCUMENT_19,FLAG_DOCUMENT_20,FLAG_DOCUMENT_21,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_YEAR
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,351000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,Laborers,1.0,2,2,WEDNESDAY,10,0,0,0,0,0,0,...,0.0205,0.0193,0.0000,0.00,reg oper account,block of flats,0.0149,"Stone, brick",No,2.0,2.0,2.0,2.0,-1134.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,1129500.0,Family,State servant,Higher education,Married,House / apartment,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,Core staff,2.0,1,1,MONDAY,11,0,0,0,0,0,0,...,0.0787,0.0558,0.0039,0.01,reg oper account,block of flats,0.0714,Block,No,1.0,0.0,1.0,0.0,-828.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,135000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,Laborers,1.0,2,2,MONDAY,9,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-815.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,297000.0,Unaccompanied,Working,Secondary / secondary special,Civil marriage,House / apartment,0.008019,-19005,-3039,-9833.0,-2437,NaN,1,1,0,1,0,0,Laborers,2.0,2,2,WEDNESDAY,17,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,0.0,2.0,0.0,-617.0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,513000.0,Unaccompanied,Working,Secondary / secondary special,Single / not married,House / apartment,0.028663,-19932,-3038,-4311.0,-3458,NaN,1,1,0,1,0,0,Core staff,1.0,2,2,THURSDAY,11,0,0,0,0,1,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,-1106.0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0.0,0.0,0.0,0.0,0.0,0.0


##Missing Values

Trying to see what should we do with the missing values.

In [0]:
def missing_value_statistics(df, head = 0, columns = False):
  # Creating table of missing values
  missing = df.isnull().sum()
  missing_percent = df.isnull().sum() * 100 / len(df)
  table = pd.concat([missing,missing_percent], axis = 1)
  table = table.rename(columns = {0:'Missing Value',1:'%'})
  table = table[table.iloc[:,1]!=0].sort_values('%',ascending=False).round(1)
  
  # Caulculating column data
  total_col = df.shape[1]
  missing_col = table.shape[0]
  missing_col_percent = missing_col * 100 / total_col
  
  # Calculating row data
  total_rows = df.shape[0]
  missing_rows = total_rows - df.dropna().shape[0]
  missing_rows_percent = missing_rows * 100 / len(df)
  
  print("Rows")
  print("-------------------------------------------------------")
  print("Total:", total_rows)
  print("Missing:", missing_rows)
  print("Percent: " + str(missing_rows_percent)[:5] + "%")
  
  print("\nCoulmns")
  print("-------------------------------------------------------")
  print("Total:", total_col)
  print("Missing:", missing_col)
  print("Percent: " + str(missing_col_percent)[:5] + "%")
  
  if(columns):
    print("\n")
    print(table.index)
  
  if(head != 0):
    print("\n")
    if(head < 0):
      print(table)
    else:
      print(table.head(head))
                             


In [8]:
missing_value_statistics(df, 15, True)

Rows
-------------------------------------------------------
Total: 307511
Missing: 298909
Percent: 97.20%

Coulmns
-------------------------------------------------------
Total: 122
Missing: 67
Percent: 54.91%


Index(['COMMONAREA_MEDI', 'COMMONAREA_AVG', 'COMMONAREA_MODE',
       'NONLIVINGAPARTMENTS_MEDI', 'NONLIVINGAPARTMENTS_MODE',
       'NONLIVINGAPARTMENTS_AVG', 'FONDKAPREMONT_MODE',
       'LIVINGAPARTMENTS_MODE', 'LIVINGAPARTMENTS_MEDI',
       'LIVINGAPARTMENTS_AVG', 'FLOORSMIN_MODE', 'FLOORSMIN_MEDI',
       'FLOORSMIN_AVG', 'YEARS_BUILD_MODE', 'YEARS_BUILD_MEDI',
       'YEARS_BUILD_AVG', 'OWN_CAR_AGE', 'LANDAREA_AVG', 'LANDAREA_MEDI',
       'LANDAREA_MODE', 'BASEMENTAREA_MEDI', 'BASEMENTAREA_AVG',
       'BASEMENTAREA_MODE', 'EXT_SOURCE_1', 'NONLIVINGAREA_MEDI',
       'NONLIVINGAREA_MODE', 'NONLIVINGAREA_AVG', 'ELEVATORS_MEDI',
       'ELEVATORS_MODE', 'ELEVATORS_AVG', 'WALLSMATERIAL_MODE',
       'APARTMENTS_MODE', 'APARTMENTS_MEDI', 'APARTMENTS_AVG',
       'ENTRANCES

As we can see we can't simply delete the ros with missing values because we lose 97% of the data if we do. It is not a good practice anyways.

Another option is to drop the columns with high percentace of missing data. However, we can't say anything about how important they are right now. Therefore, it is better to keep them in.

We can use models that can handle missing values but we prefer to fill them in order to be able to use other models.

#Preprocessing

*   Handling Missing Values
*   Encoding categorical Variables
*   Scaling



In [9]:
# Encoding the data
df = pd.get_dummies(df)
df.head()

,SK_ID_CURR,TARGET,CNT_CHILDREN,AMT_INCOME_TOTAL,AMT_CREDIT,AMT_ANNUITY,AMT_GOODS_PRICE,REGION_POPULATION_RELATIVE,DAYS_BIRTH,DAYS_EMPLOYED,DAYS_REGISTRATION,DAYS_ID_PUBLISH,OWN_CAR_AGE,FLAG_MOBIL,FLAG_EMP_PHONE,FLAG_WORK_PHONE,FLAG_CONT_MOBILE,FLAG_PHONE,FLAG_EMAIL,CNT_FAM_MEMBERS,REGION_RATING_CLIENT,REGION_RATING_CLIENT_W_CITY,HOUR_APPR_PROCESS_START,REG_REGION_NOT_LIVE_REGION,REG_REGION_NOT_WORK_REGION,LIVE_REGION_NOT_WORK_REGION,REG_CITY_NOT_LIVE_CITY,REG_CITY_NOT_WORK_CITY,LIVE_CITY_NOT_WORK_CITY,EXT_SOURCE_1,EXT_SOURCE_2,EXT_SOURCE_3,APARTMENTS_AVG,BASEMENTAREA_AVG,YEARS_BEGINEXPLUATATION_AVG,YEARS_BUILD_AVG,COMMONAREA_AVG,ELEVATORS_AVG,ENTRANCES_AVG,FLOORSMAX_AVG,...,ORGANIZATION_TYPE_Police,ORGANIZATION_TYPE_Postal,ORGANIZATION_TYPE_Realtor,ORGANIZATION_TYPE_Religion,ORGANIZATION_TYPE_Restaurant,ORGANIZATION_TYPE_School,ORGANIZATION_TYPE_Security,ORGANIZATION_TYPE_Security Ministries,ORGANIZATION_TYPE_Self-employed,ORGANIZATION_TYPE_Services,ORGANIZATION_TYPE_Telecom,ORGANIZATION_TYPE_Trade: type 1,ORGANIZATION_TYPE_Trade: type 2,ORGANIZATION_TYPE_Trade: type 3,ORGANIZATION_TYPE_Trade: type 4,ORGANIZATION_TYPE_Trade: type 5,ORGANIZATION_TYPE_Trade: type 6,ORGANIZATION_TYPE_Trade: type 7,ORGANIZATION_TYPE_Transport: type 1,ORGANIZATION_TYPE_Transport: type 2,ORGANIZATION_TYPE_Transport: type 3,ORGANIZATION_TYPE_Transport: type 4,ORGANIZATION_TYPE_University,ORGANIZATION_TYPE_XNA,FONDKAPREMONT_MODE_not specified,FONDKAPREMONT_MODE_org spec account,FONDKAPREMONT_MODE_reg oper account,FONDKAPREMONT_MODE_reg oper spec account,HOUSETYPE_MODE_block of flats,HOUSETYPE_MODE_specific housing,HOUSETYPE_MODE_terraced house,WALLSMATERIAL_MODE_Block,WALLSMATERIAL_MODE_Mixed,WALLSMATERIAL_MODE_Monolithic,WALLSMATERIAL_MODE_Others,WALLSMATERIAL_MODE_Panel,"WALLSMATERIAL_MODE_Stone, brick",WALLSMATERIAL_MODE_Wooden,EMERGENCYSTATE_MODE_No,EMERGENCYSTATE_MODE_Yes
0,100002,1,0,202500.0,406597.5,24700.5,351000.0,0.018801,-9461,-637,-3648.0,-2120,NaN,1,1,0,1,1,0,1.0,2,2,10,0,0,0,0,0,0,0.083037,0.262949,0.139376,0.0247,0.0369,0.9722,0.6192,0.0143,0.00,0.0690,0.0833,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,1,0
1,100003,0,0,270000.0,1293502.5,35698.5,1129500.0,0.003541,-16765,-1188,-1186.0,-291,NaN,1,1,0,1,1,0,2.0,1,1,11,0,0,0,0,0,0,0.311267,0.622246,NaN,0.0959,0.0529,0.9851,0.7960,0.0605,0.08,0.0345,0.2917,...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,1,0
2,100004,0,0,67500.0,135000.0,6750.0,135000.0,0.010032,-19046,-225,-4260.0,-2531,26.0,1,1,1,1,1,0,1.0,2,2,9,0,0,0,0,0,0,NaN,0.555912,0.729567,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,100006,0,0,135000.0,312682.5,29686.5,297000.0,0.008019,-19005,-3039,-9833.0,-2437,NaN,1,1,0,1,0,0,2.0,2,2,17,0,0,0,0,0,0,NaN,0.650442,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,100007,0,0,121500.0,513000.0,21865.5,513000.0,0.028663,-19932,-3038,-4311.0,-3458,NaN,1,1,0,1,0,0,1.0,2,2,11,0,0,0,0,1,1,NaN,0.322738,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [0]:
# Seperating data and target
labels = df['TARGET']
train = df.drop(columns = ['TARGET'])

In [0]:
# enabling Iterative Imputer because it is experimental
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

# Still will test with simple imputer
from sklearn.impute import SimpleImputer

simp_imputer = SimpleImputer(missing_values = np.nan, strategy = "median")
simp_imputed_train = simp_imputer.fit_transform(train)

In [12]:
iter_imputer = IterativeImputer(max_iter = 10, n_nearest_features = 15,
                               initial_strategy = "median", verbose = 2)
iter_imputed_train = iter_imputer.fit_transform(train)

[IterativeImputer] Completing matrix with shape (307511, 245)
[IterativeImputer] Ending imputation round 1/10, elapsed time 14.21
[IterativeImputer] Ending imputation round 2/10, elapsed time 28.45
[IterativeImputer] Early stopping criterion reached.


Using iteretive imputer, even it is experimental, to see if it will perform better than replacing missing values with median.

In [0]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range = (0,1))

# Scaling values between 0 and 1
simp_imputed_train = scaler.fit_transform(simp_imputed_train)
iter_imputed_train = scaler.fit_transform(iter_imputed_train)

In [0]:
from sklearn.model_selection import train_test_split

# Spliting the datas
X_train_s, X_test_s, y_train_s, y_test_s = train_test_split(simp_imputed_train, labels, test_size = 0.2, random_state = 1)
X_train_i, X_test_i, y_train_i, y_test_i = train_test_split(iter_imputed_train, labels, test_size = 0.2, random_state = 1)

#Logistic Regression

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Train simple imputed
simp_log_reg = LogisticRegression().fit(X_train_s, y_train_s)

In [16]:
# Predict simple imputed
accuracy_score(y_test_s, simp_log_reg.predict(X_test_s))

0.9192884899923581

In [0]:
# Train iteratively imputed
iter_log_reg = LogisticRegression().fit(X_train_i, y_train_i)

In [18]:
# Predict iteratively imputed
accuracy_score(y_test_i, iter_log_reg.predict(X_test_i))

0.9196461961205145

As it can be seen, using iterative imputer improved the performance but it is not a significant difference. Will be using only iterative imputed data from now on since it will take twice as much time as it takes otherwise.

Since the accuracy is high we will use cross validation to see if it is overfitting or not.

In [19]:
from sklearn.model_selection import cross_val_score

# Cross validate
cv_scores = cross_val_score(iter_log_reg, iter_imputed_train, labels, cv = 10)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  Futur

In [20]:
cv_scores

array([0.91941988, 0.91958247, 0.91932232, 0.91899714, 0.91922477,
       0.91938474, 0.9195122 , 0.91938211, 0.91960976, 0.91960976])

In [21]:
print("Accuracy: %0.4f (+/- %0.4f)" % (cv_scores.mean(), cv_scores.std() * 2))

Accuracy: 0.9194 (+/- 0.0004)


It seems our model is not overfitting.

#Random Forest Classification

In [22]:
from sklearn.ensemble import RandomForestClassifier

# Initialise the random forest classifier
rand_forest = RandomForestClassifier(n_estimators = 100, random_state = 50, 
                                     verbose = 1, n_jobs = -1)

# Train
rand_forest.fit(X_train_i, y_train_i)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  46 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:  2.4min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=50, verbose=1,
                       warm_start=False)

In [23]:
# Predict
accuracy_score(y_test_i,rand_forest.predict(X_test_i))

[Parallel(n_jobs=2)]: Using backend ThreadingBackend with 2 concurrent workers.
[Parallel(n_jobs=2)]: Done  46 tasks      | elapsed:    0.8s
[Parallel(n_jobs=2)]: Done 100 out of 100 | elapsed:    1.7s finished


0.9191909337755881

#XGBoost

In [24]:
from xgboost import XGBClassifier

# Initialise XGBoost classifier
XGB = XGBClassifier(max_depth = 6, learning_rate = 0.1,
                   n_estimators = 100, verbosity = 1)

# Train
XGB.fit(X_train_i, y_train_i)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [25]:
accuracy_score(y_test_i, XGB.predict(X_test_i))

0.9196461961205145

The results we get from the models are very close to each other. Since we got the best result from XGBoost when we train and test with the same data, we will use it while submitting the results to competition.

#Preparing test data for submition

We will apply the same process to test data as well.

In [26]:
!mv competitions/home-credit-default-risk/application_test.csv.zip /content/
!unzip application_test.csv.zip
!rm application_test.csv.zip

Archive:  application_test.csv.zip
  inflating: application_test.csv    


Here we will read both test and train data again and apply the same process on both of them because there are missing columns on test data and we need to align them.

In [0]:
test = pd.read_csv("application_test.csv")
df = pd.read_csv("application_train.csv")
ids = test['SK_ID_CURR']

In [37]:
# Encoding the data
test = pd.get_dummies(test)
train = pd.get_dummies(df)

labels = train['TARGET']
# Align test and training data.
# Only keeps the columns that both tables have

train, test = train.align(test, join = 'inner', axis = 1)

train.shape, test.shape

((307511, 242), (48744, 242))

In [38]:
# Imputing missing values
test = iter_imputer.fit_transform(test)
train = iter_imputer.fit_transform(train)

# Scaling values between 0 and 1
test = scaler.fit_transform(test)
train = scaler.fit_transform(train)

/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2530: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[:, None]
/usr/local/lib/python3.6/dist-packages/numpy/lib/function_base.py:2531: RuntimeWarning: invalid value encountered in true_divide
  c /= stddev[None, :]


[IterativeImputer] Completing matrix with shape (48744, 242)
[IterativeImputer] Ending imputation round 1/10, elapsed time 1.62
[IterativeImputer] Ending imputation round 2/10, elapsed time 3.25
[IterativeImputer] Ending imputation round 3/10, elapsed time 4.82
[IterativeImputer] Early stopping criterion reached.
[IterativeImputer] Completing matrix with shape (307511, 242)
[IterativeImputer] Ending imputation round 1/10, elapsed time 14.61
[IterativeImputer] Ending imputation round 2/10, elapsed time 29.28
[IterativeImputer] Early stopping criterion reached.


In [39]:
# Initialise XGBoost classifier
XGB = XGBClassifier(max_depth = 6, learning_rate = 0.1,
                   n_estimators = 100, verbosity = 1)
# We will train the model with whole train data for better accuracy
XGB.fit(train, labels)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [0]:
results = XGB.predict(test)

In [63]:
results.shape, df.shape

((48744,), (307511, 122))

In [0]:
submition = pd.DataFrame({'SK_ID_CURR': ids, 'TARGET': results})

In [73]:
submition.head(20)

,SK_ID_CURR,TARGET
0,100001,0
1,100005,0
2,100013,0
3,100028,0
4,100038,0
5,100042,0
6,100057,0
7,100065,0
8,100066,0
9,100067,0


In [0]:
submition.to_csv(r'submition.csv')